# Working with Annual Averages in Cal-Adapt API

In [2]:
import requests 
import json
headers = {'ContentType': 'json'}

api = 'http://api.cal-adapt.org/api'
resource = 'series'

### Get a list of annual timeseries from Cal-Adapt API for maximum temperature

In [3]:
search_str = 'yearly+average+maximum+temperature'
params = {'name': search_str, 'pagesize': 100}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
url = api + '/' + resource + '/' + '?' + params_str
url

'http://api.cal-adapt.org/api/series/?name=yearly+average+maximum+temperature&pagesize=100'

In [4]:
timeseries = None
response = requests.get(url, headers=headers)
if response.status_code == requests.codes.ok:
    data = response.json()
    timeseries = data['results']
    for item in timeseries:
        print(item['name'])

yearly average maximum temperature ACCESS1-0 modeled historical
yearly average maximum temperature ACCESS1-0 RCP 4.5
yearly average maximum temperature ACCESS1-0 RCP 8.5
yearly average maximum temperature CCSM4 modeled historical
yearly average maximum temperature CCSM4 RCP 4.5
yearly average maximum temperature CCSM4 RCP 8.5
yearly average maximum temperature CESM1-BGC modeled historical
yearly average maximum temperature CESM1-BGC RCP 4.5
yearly average maximum temperature CESM1-BGC RCP 8.5
yearly average maximum temperature CMCC-CMS modeled historical
yearly average maximum temperature CMCC-CMS RCP 4.5
yearly average maximum temperature CMCC-CMS RCP 8.5
yearly average maximum temperature CNRM-CM5 modeled historical
yearly average maximum temperature CNRM-CM5 RCP 4.5
yearly average maximum temperature CNRM-CM5 RCP 8.5
yearly average maximum temperature CanESM2 modeled historical
yearly average maximum temperature CanESM2 RCP 4.5
yearly average maximum temperature CanESM2 RCP 8.5
year

### Get url for model HadGEM2-ES and RCP 8.5

In [7]:
ts = [x for x in timeseries if (x['name'].find('Livneh') > -1)]
model_ts = ts[0]
model_ts

{'begin': '1950-01-01T00:00:00Z',
 'end': '2013-01-01T00:00:00Z',
 'name': 'yearly average maximum temperature Livneh',
 'rasters': ['http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1950/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1951/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1952/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1953/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1954/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1955/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1956/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1957/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1958/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1959/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1960/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1961/',
  'http://api.cal-adapt.org/api/rstores/tasmax_year_livneh_1962/'

### Setup parameters for requesting data

In [5]:
# Period
startYear = 1961
endYear = 1990
period = str(startYear) + '-01-01/' + str(endYear) + '-12-31'
# Alternatively get complete timeseries and filter dates later on
period = 'rasters'

In [6]:
# Point location in geojson format
lat = 38.5938
lng = -121.4687
point = '{"type":"Point","coordinates":[' + str(lng) + ',' + str(lat) + ']}'

In [7]:
# Pagesize - number of records returned, default is 10
pagesize = 100

In [8]:
params = {'pagesize': pagesize, 'g': point}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())

### Request annual averages for a timeseries and location

In [9]:
data = None

url = model_ts['url'] +  period + '/?' + params_str
print(url)

response = requests.get(url, headers=headers)
if response.status_code == requests.codes.ok:
    json = response.json()
    data = json['results']
    print(data)

http://api.cal-adapt.org/api/series/tasmax_year_HadGEM2-ES_rcp85/rasters/?pagesize=100&g={"type":"Point","coordinates":[-121.4687,38.5938]}
[{'id': 12320, 'tileurl': 'http://api.cal-adapt.org/tiles/tasmax_year_HadGEM2-ES_rcp85_2006/{z}/{x}/{y}.png', 'url': 'http://api.cal-adapt.org/api/rstores/tasmax_year_HadGEM2-ES_rcp85_2006/', 'image': 297.0770568847656, 'width': 179, 'height': 195, 'geom': {'type': 'Polygon', 'coordinates': [[[-124.5625, 31.5625], [-113.375, 31.5625], [-113.375, 43.75], [-124.5625, 43.75], [-124.5625, 31.5625]]]}, 'event': '2006-01-01', 'srs': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433],AUTHORITY["EPSG","4326"]]', 'minval': 277.29461669921875, 'maxval': 306.6802673339844, 'nodata': 1.0000000150474662e+30, 'xpixsize': 0.0625, 'ypixsize': -0.0625, 'name': 'yearly average maximum temperature HadGEM2-ES RCP 8.5', 'slug': 'tasmax_year

### Create pandas dataframe from data.

In [10]:
import pandas as pd

df = pd.DataFrame.from_records([s for s in data])
# Change format of `event` field from string to datetime
df['event'] = pd.to_datetime(df['event'], format='%Y-%m-%d')
# Set event field as index
df = df.set_index(['event'])
df.head()

,geom,height,id,image,maxval,minval,name,nodata,slug,srs,tileurl,units,url,width,xpixsize,ypixsize
event,,,,,,,,,,,,,,,,
2006-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12320,297.077057,306.680267,277.294617,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2006,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2007-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12321,298.839325,306.515289,279.461761,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2007,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2008-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12322,297.425262,306.392303,277.135956,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2008,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2009-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12323,297.414825,306.001038,277.822418,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2009,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2010-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12324,296.677643,305.385651,277.153900,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2010,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625


### Filter dataframe for years 2031 - 2060

In [11]:
filtered = df.ix['2031-01-01':'2060-01-01']
filtered.head()

,geom,height,id,image,maxval,minval,name,nodata,slug,srs,tileurl,units,url,width,xpixsize,ypixsize
event,,,,,,,,,,,,,,,,
2031-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12345,298.168671,307.710938,278.883209,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2031,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2032-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12346,299.460297,308.421173,279.548584,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2032,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2033-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12347,298.551941,306.917236,279.081390,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2033,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2034-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12348,299.920319,308.820526,280.245483,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2034,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625
2035-01-01,"{'type': 'Polygon', 'coordinates': [[[-124.562...",195,12349,299.189911,308.117859,278.713013,yearly average maximum temperature HadGEM2-ES ...,1.000000e+30,tasmax_year_HadGEM2-ES_rcp85_2035,"GEOGCS[""WGS 84"",DATUM[""WGS_1984"",SPHEROID[""WGS...",http://api.cal-adapt.org/tiles/tasmax_year_Had...,K,http://api.cal-adapt.org/api/rstores/tasmax_ye...,179,0.0625,-0.0625


#### The `image` field contains the value for climate variable, in this case maximum temeperature. The `units` field indicates units for the data, in this case Kelvin. Calculate average temperature.

In [12]:
avgK = filtered['image'].mean()
avgK

299.5190012613932

Convert to degrees Fahrenheit

In [13]:
avgF = (avgK * 9/5) - 459.67
print(model_ts['name'])
print('2031-2060 Average -', round(avgF, 1), 'degrees Fahrenheit', sep = ' ')

yearly average maximum temperature HadGEM2-ES RCP 8.5
2031-2060 Average - 79.5 degrees Fahrenheit


### Note:
* The Livneh timeseries (1950 -2013) contains the historical observed data 
* The modeled projections are available for two different periods:
    * One historical hindcast period from 1950-2005
    * Two future scenarios RCP 4.5 and RCP 8.5 from 2006–2100 (although some models stop in 2099).
* Additionally, an envelope of modeled variability was generated by selecting the highest and lowest values from annual averages of all 32 LOCA downscaled climate models. This envelope is available as two different timeseries:
    * Modelled Variability Envelope (Max)
    * Modelled Variability Envelope (Min)

In [14]:
search_str = 'allmodels+maximum+temperature'
params = {'name': search_str, 'pagesize': 10}
params_str = "&".join("%s=%s" % (k,v) for k,v in params.items())
url = api + '/' + resource + '/' + '?' + params_str
response = requests.get(url, headers=headers)
if response.status_code == requests.codes.ok:
    data = response.json()
    timeseries = data['results']
    for item in timeseries:
        print(item['name'])

maximum allmodels maximum temperature historical
maximum allmodels maximum temperature RCP 4.5
maximum allmodels maximum temperature RCP 8.5
minimum allmodels maximum temperature historical
minimum allmodels maximum temperature RCP 4.5
minimum allmodels maximum temperature RCP 8.5
